In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import os


def newfile(file):
  with open(file, 'w') as fp:
    fp.writelines('{"data":[]}')
    pass

def write_json(new_data,file):
  with open(file, 'r+') as file:
    file_data = json.load(file)
    file_data['data'].append(new_data)
    file.seek(0)
    json.dump(file_data, file, indent=4)


def products_scraper(file):
  baseurl = 'https://www.bplmedicaltechnologies.com'
  url1 = '/products/'
  r = requests.get(url=baseurl+url1)
  data = r.content
  products = []
  soup = BeautifulSoup(data, "html.parser")
  productsoup = soup.find('ul', {'class': "prd-lst-ul contentscroll2"})
  for product in productsoup.find_all('div', {'class': "white-box"}):
    image = product.find('img').get('src')
    title = product.find('h3', {'class': "prdct-title"}).text
    data = [title, image]
    write_json(data,file)

def subproducts_scraper(file):
  baseurl = 'https://www.bplmedicaltechnologies.com'
  url1 = '/products/'
  r = requests.get(url=baseurl+url1)
  data = r.content
  products = []
  soup = BeautifulSoup(data, "html.parser")
  for link in soup.findAll('a'):
    rawlink = link.get('href')
    if ('/products/' in rawlink):
      rawlink = rawlink.split('/')[2]
      if rawlink != '':
        rawlink = url1+rawlink+'/'
        if (rawlink not in products):
          products.append(rawlink)

  for i in range(len(products)):
    subproducts = []
    imagegallery = []
    featuresls = []
    namels = []
    url2 = baseurl+products[i]
    r = requests.get(url=url2)
    data = r.content
    soup = BeautifulSoup(data, "html.parser")
    for link in soup.findAll('a'):
      rawlink = link.get('href')
      if ('/product-details/' in rawlink):
        if rawlink != '':
          rawlink = baseurl+rawlink
          if (rawlink not in subproducts):
            subproducts.append(rawlink)

    for j in range(len(subproducts)):
      url = subproducts[j]
      r = requests.get(url=url)
      data = r.content
      image = []
      featureli = []
      name = ''
      cate = ''
      soup = BeautifulSoup(data, "html.parser")
      try:
        souptumbnail = soup.find('div', {'class': "pd-intr-image"})
        for thumbnail in souptumbnail.findAll('img'):
          rawlink = thumbnail.get('src')
          if 'https://www.bplmedicaltechnologies.com/' in rawlink:
            image.append(rawlink)
      except:
        pass
      try:
        soupimg = soup.find('ul', {'id': "gallery-flex"})
        for link in soupimg.findAll('img'):
          rawlink = link.get('src')
          if 'https://www.bplmedicaltechnologies.com/' in rawlink:
            if rawlink not in image:
              image.append(rawlink)
      except:
        pass
      soupcat = soup.find('div', {'class': "navigation"})
      try:
        soupname = soup.find('div', {'class': "new-layout-heading"})
        name = soupname.text.replace("\n", '')
      except:
        soupname = soup.find('div', {'class': "navigation"})
        try:
          name = soupcat.text.split("//")[4].replace('\r\n', '')
        except:
          name = soupcat.text.split("//")[2].replace('\r\n', '')

      soupfeatures = soup.find('div', {'class': "proct-feature-list"})
      for link in soupfeatures.find_all('li'):
        feature = link.text
        feature = re.sub('[^a-zA-Z0-9 : - /\.]', '', feature)
        featureli.append(feature)
      cate = soupcat.text.split("//")[1]
      name = re.sub(' +', ' ', name)
      cate = re.sub(' +', ' ', cate).replace('\r\n', '').strip()
      cate = cate.replace(' ', '')
      imagegallery.append(image)
      featuresls.append(featureli)
      namels.append(name)
      j += 1
    data = [[a, b, c] for a, b, c in zip(imagegallery, namels, featuresls)]
    data = {cate: data}
    write_json(data, file)
    i += 1


if __name__ == "__main__":
  newfile("subproducts.json")
  subproducts_scraper("subproducts.json")
  newfile("products.json")
  products_scraper("products.json")
